<a href="https://colab.research.google.com/github/dasmiq/CS6120-PS4/blob/master/CrossLanguageEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

We have mentioned, and will discuss in more detail this week, how we can reduce the dimensionality of word representations from their original vectors space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classifiers, parsers, and other inference models that use those embeddings.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [1]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the Shakespeare plays we used for the first assignment.

In [2]:
#!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [3]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [4]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.8424739837646484),
 ('york', 0.7672518491744995),
 ('warwick', 0.7594640254974365),
 ('clarence', 0.7455054521560669),
 ('gloucester', 0.7409666776657104),
 ('prince', 0.74066561460495),
 ('suffolk', 0.7236307859420776),
 ('duke', 0.7061275243759155),
 ('cardinal', 0.6962414979934692),
 ('margaret', 0.6912989616394043)]

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [5]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.9550749659538269),
 ('emilia', 0.9382157325744629),
 ('iago', 0.9354672431945801),
 ('ham', 0.9116354584693909),
 ('cleopatra', 0.9110197424888611),
 ('cressida', 0.9105851650238037),
 ('proteus', 0.903400719165802),
 ('celia', 0.9003647565841675),
 ('imogen', 0.8995494842529297),
 ('cassio', 0.8943049907684326)]

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [6]:
model.wv.similarity('othello', 'desdemona')

0.955075

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [7]:
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  return np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.955075

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [10]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.pt.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ru.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.zh.vec

9% 17.2M 2s
 13000K .......... .......... .......... .......... .......... 19% 28.4M 2s
 13050K .......... .......... .......... .......... .......... 19% 62.7M 2s
 13100K .......... .......... .......... .......... .......... 19% 26.9M 2s
 13150K .......... .......... .......... .......... .......... 19% 25.5M 2s
 13200K .......... .......... .......... .......... .......... 20% 28.4M 2s
 13250K .......... .......... .......... .......... .......... 20% 30.1M 2s
 13300K .......... .......... .......... .......... .......... 20% 16.6M 2s
 13350K .......... .......... .......... .......... .......... 20% 40.5M 2s
 13400K .......... .......... .......... .......... .......... 20% 36.6M 2s
 13450K .......... .......... .......... .......... .......... 20% 30.8M 2s
 13500K .......... .......... .......... .......... .......... 20% 23.0M 2s
 13550K .......... .......... .......... .......... .......... 20% 27.1M 2s
 13600K .......... .......... .......... .......... .......... 20% 19.4M 2s


We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [8]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname, encoding="utf8")] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
ruvec = ftvectors('30k.ru.vec')
# esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [9]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  similarity = 0
  mostSimilar = ''
  for vec_ in vecDict:
    cosim_val = cosim(vec, vecDict[vec_])
    if cosim_val > similarity:
      similarity = cosim_val
      mostSimilar = vec_
      
  
  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.593718413875964),
 ('matrice', 0.5088361302065517),
 ('physique', 0.4555543434796394),
 ('fermentation', 0.3504105196166514)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [18]:
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
links = [s.split() for s in open('links.tab', encoding="utf8")]

This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [19]:
links[302]

['academy', 'fr', 'académie']

In [20]:
def calcAccuracy(links, vd1, vd2):
  baseline = len([link for link in links if link[0] == link[2]])/len(links)
  correct = 0
  for link in links:
    if link[2] == mostSimilar(vd1[link[0]], vd2)[0]:
      correct += 1
  accuracy = correct/len(links)
  return (baseline, accuracy)

In [21]:
links_fr = [link for link in links if link[1] == "fr"]
len(links_fr)

9869

In [22]:
links_ru = [link for link in links if link[1] == "ru"]
len(links_ru)

8523

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [23]:
baselineAccuracy, accuracy = calcAccuracy(links_fr, envec, frvec)
print(f"Accuracy: {accuracy}, Baseline: {baselineAccuracy}")

Accuracy: 0.5359205593271862, Baseline: 0.6742324450298915


**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [40]:
## TODO: Compute English-X Wikipedia retrieval accuracy.
baselineAccuracyRU, accuracyRU = calcAccuracy(links_ru, envec, ruvec)
print(f"Accuracy: {accuracyRU}, Baseline: {baselineAccuracyRU}")

Accuracy: 0.19183386131643787, Baseline: 0.07121905432359497


Further evaluation, if you are interested, could involve looking at the $k$ nearest neighbors of each English term to compute "recall at 10" or "mean reciprocal rank at 10".

In [15]:
def k_nn(vec, vecDict, k):
    cosimDict = {k: cosim(v, vec) for k, v in vecDict.items()} # first calculate all the cosims
    # sort the dict
    sorted_temp = sorted(cosimDict.items(), key=lambda item: item[1])
    knn = sorted_temp[-k:] # get the k highest values (this is sorted by lowest first)
    return [s[0] for s in knn]

In [16]:
print(k_nn(envec["computer"], frvec, 5))

['informaticien', 'informatiques', 'ordinateurs', 'ordinateur', 'informatique']


In [37]:
def recallAtK(links, v1, v2, k):
    len_links = len(links)
    score = 0
    for link in links:
        try:
            k_nn_list = k_nn(v1[link[0]], v2, k)
            k_nn_list.reverse()
            score += 1/(1+ k_nn_list.index(link[2]))
        except ValueError:
            score += 0
    return score/len_links

In [38]:
recallAt10_En_Fr = recallAtK(links_fr, envec, frvec, 10)
print(recallAt10_En_Fr)

0.5646970954423574


In [39]:
recallAt10_En_Ru = recallAtK(links_ru, envec, ruvec, 10)
print(recallAt10_En_Ru)
# this just needs to be run again accidently ran it with frenk links OOPS

0.010745327279423948
